In [3]:
import pandas as pd
import csv
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
!pip install Sastrawi
!pip install tensorflow==2.12.0
import tensorflow.keras
from tensorflow import keras
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, LSTM
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.15.0
    Uninstalling wrapt-1.15.0:
      Successfully uninstalled wrapt-1.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  

In [ ]:
df = pd.read_csv('comments_data.csv')

In [ ]:
df2 = pd.read_csv('comments_data_2.csv')

In [ ]:
df2.shape

(7200, 1)

In [ ]:
df = df.append(df2)

<ipython-input-87-fb89c00654ab>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)


In [107]:
df['Comment'] = df['Comment'].astype(str)

NameError: ignored

In [ ]:
df.head(5)

,Comment
0,Kemenangan SEMPURNA diberikan Timnas Indonesia...
1,Mohon do'anya semoga dagangan minyak saya lari...
2,Brunei adalah tim yang menjunjung tinggi sport...
3,"Kalian boleh kalah dlm hal olahraga, tp Negara..."
4,STY sudah membentuk timnas Indonesia saat ini ...


In [ ]:
df = df.sample(frac = 1) # shuffle dataset so that the training and test data are randomized by video categories

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.head(10)

,Comment
0,Bner dh fiki keren
1,"@deltaacro1211 bunga nya gak berguna, di bil..."
2,​ @remanrhn waduh menn
3,Semangat bang
4,Agak laen emang kalau bang RemanRHN upload video💀
5,Orang indo makan singkong gk bisa pastur tingg...
6,Gemesss bangt jejeeeee 😂😂😂😂
7,"Suaranya bang indra jegel ngga bermasalah kok,..."
8,❤.🇮🇩 koh rudi ikut menyambut.presiden bpk.joko...
9,😂😂😆😆😆😆....saaaatttt.....


**Preprocess Data**

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.dropna(inplace=True)

In [4]:
# Some functions for preprocessing text

def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # remove mentions
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # remove hashtag
    text = re.sub(r'RT[\s]', '', text) # remove RT
    text = re.sub(r"http\S+", '', text) # remove link
    text = re.sub(r'[0-9]+', '', text) # remove numbers

    text = text.replace('\n', ' ') # replace new line into space
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove all punctuations
    text = text.strip(' ') # remove characters space from both left and right text
    return text

def casefoldingText(text): # Converting all the characters in a text into lower case
    text = text.lower()
    return text

def tokenizingText(text): # Tokenizing or splitting a string, text into a list of tokens
    text = word_tokenize(text)
    return text

def filteringText(text): # Remove stopwors in a text
    listStopwords = set(stopwords.words('indonesian'))
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = [stemmer.stem(word) for word in text]
    return text

def toSentence(list_words): # Convert list of words into sentence
    sentence = ' '.join(word for word in list_words)
    return sentence

In [108]:
df['Comment_Clean'] = df['Comment'].apply(cleaningText)
df['Comment_Clean'] = df['Comment_Clean'].apply(casefoldingText)
df.drop(['Comment'], axis = 1, inplace = True)

df['Comment_Preprocessed'] = df['Comment_Clean'].apply(tokenizingText)
df['Comment_Preprocessed'] = df['Comment_Preprocessed'].apply(filteringText)
df['Comment_Preprocessed'] = df['Comment_Preprocessed'].apply(stemmingText)

df.drop_duplicates(subset = 'Comment_Clean', inplace = True)

df.to_csv(r'comments_data_clean.csv', index = False, header = True, index_label=None)

NameError: ignored

In [ ]:
df.shape

(8355, 2)

In [8]:
comments = pd.read_csv('comments_data_clean.csv', lineterminator='\n')
comments

,Comment_Clean,Comment_Preprocessed\r
0,efek ganti pelatih persiapan mepet dan juga pe...,"['efek', 'ganti', 'latih', 'siap', 'mepet', 'm..."
1,keren indonesia,"['keren', 'indonesia']\r"
2,udah jadian aja deh jeje ma ririn 😭😭,"['udah', 'jadi', 'aja', 'deh', 'jeje', 'ma', '..."
3,semangat timnas indonesia jangan pernah mereme...,"['semangat', 'timnas', 'indonesia', 'remeh', '..."
4,chemistry surya jegel emang the best👍,"['chemistry', 'surya', 'jegel', 'emang', 'the'..."
...,...,...
8350,buat pach sekarang emng memaksa xp gabakala...,"['pach', 'emng', 'paksa', 'xp', 'gabakalan', '..."
8351,thank you rudy saya salut dengan kebanggaan ru...,"['thank', 'you', 'rudy', 'salut', 'bangga', 'r..."
8352,aransemen dc memang pancen wenakk gaaass mas d...,"['aransemen', 'dc', 'pancen', 'wenakk', 'gaaas..."
8353,wonderful indonesia 🇮🇩❤️,"['wonderful', 'indonesia', '']\r"


In [9]:
comments=comments.replace({'\r': ''}, regex=True)

In [10]:
comments.rename(columns = {'Comment_Preprocessed\r':'Comment_Preprocessed'}, inplace = True)

In [11]:
comments

,Comment_Clean,Comment_Preprocessed
0,efek ganti pelatih persiapan mepet dan juga pe...,"['efek', 'ganti', 'latih', 'siap', 'mepet', 'm..."
1,keren indonesia,"['keren', 'indonesia']"
2,udah jadian aja deh jeje ma ririn 😭😭,"['udah', 'jadi', 'aja', 'deh', 'jeje', 'ma', '..."
3,semangat timnas indonesia jangan pernah mereme...,"['semangat', 'timnas', 'indonesia', 'remeh', '..."
4,chemistry surya jegel emang the best👍,"['chemistry', 'surya', 'jegel', 'emang', 'the'..."
...,...,...
8350,buat pach sekarang emng memaksa xp gabakala...,"['pach', 'emng', 'paksa', 'xp', 'gabakalan', '..."
8351,thank you rudy saya salut dengan kebanggaan ru...,"['thank', 'you', 'rudy', 'salut', 'bangga', 'r..."
8352,aransemen dc memang pancen wenakk gaaass mas d...,"['aransemen', 'dc', 'pancen', 'wenakk', 'gaaas..."
8353,wonderful indonesia 🇮🇩❤️,"['wonderful', 'indonesia', '']"


In [12]:
for i, text in enumerate(comments['Comment_Preprocessed']):
    comments['Comment_Preprocessed'][i] = comments['Comment_Preprocessed'][i].replace("'", "")\
                                            .replace(',','').replace(']','').replace('[','')
    list_words=[]
    for word in comments['Comment_Preprocessed'][i].split():
        list_words.append(word)

    comments['Comment_Preprocessed'][i] = list_words

comments

,Comment_Clean,Comment_Preprocessed
0,efek ganti pelatih persiapan mepet dan juga pe...,"[efek, ganti, latih, siap, mepet, main, remeh,..."
1,keren indonesia,"[keren, indonesia]"
2,udah jadian aja deh jeje ma ririn 😭😭,"[udah, jadi, aja, deh, jeje, ma, ririn]"
3,semangat timnas indonesia jangan pernah mereme...,"[semangat, timnas, indonesia, remeh, lawan, pe..."
4,chemistry surya jegel emang the best👍,"[chemistry, surya, jegel, emang, the, best]"
...,...,...
8350,buat pach sekarang emng memaksa xp gabakala...,"[pach, emng, paksa, xp, gabakalan, mvp, susah]"
8351,thank you rudy saya salut dengan kebanggaan ru...,"[thank, you, rudy, salut, bangga, rudy, presid..."
8352,aransemen dc memang pancen wenakk gaaass mas d...,"[aransemen, dc, pancen, wenakk, gaaass, mas, den]"
8353,wonderful indonesia 🇮🇩❤️,"[wonderful, indonesia]"


**Determining comments sentiment polarity with InSet (Indonesia Sentiment Lexicon)**

In [14]:
pos_lexicon = dict()
neg_lexicon = dict()

import csv
with open('positive.tsv') as tsv_file:
    reader = csv.reader(tsv_file, delimiter='\t')
    next(reader)
    for word, weight in reader:
        pos_lexicon[word] = int(weight)

with open('negative.tsv') as tsv_file:
    reader = csv.reader(tsv_file, delimiter='\t')
    next(reader)
    for word, weight in reader:
        neg_lexicon[word] = int(weight)

In [15]:
# Function to determine sentiment polarity of tweets
def sentiment_analysis_lexicon_indonesia(text):
    score = 0
    for word in text:
        if (word in pos_lexicon):
            score = score + pos_lexicon[word]
    for word in text:
        if (word in neg_lexicon):
            score = score + neg_lexicon[word]
    polarity=''
    if (score > 0):
        polarity = 'positive'
    elif (score < 0):
        polarity = 'negative'
    else:
        polarity = 'neutral'
    return score, polarity

In [16]:
sentiment_polarity_scores = comments['Comment_Preprocessed'].apply(sentiment_analysis_lexicon_indonesia)
sentiment_polarity_scores

0        (5, positive)
1         (0, neutral)
2         (0, neutral)
3         (0, neutral)
4         (0, neutral)
             ...      
8350    (-7, negative)
8351    (21, positive)
8352      (0, neutral)
8353      (0, neutral)
8354      (0, neutral)
Name: Comment_Preprocessed, Length: 8355, dtype: object

In [17]:
sentiment_polarity_scores = list(zip(*sentiment_polarity_scores))

In [18]:
comments['Polarity_Score'] = sentiment_polarity_scores[0]
comments['Sentiment'] = sentiment_polarity_scores[1]
#comments.to_csv(r'comments_data_clean_polarity.csv', index=False, header=True, index_label=True)

In [19]:
comments.head(20)

,Comment_Clean,Comment_Preprocessed,Polarity_Score,Sentiment
0,efek ganti pelatih persiapan mepet dan juga pe...,"[efek, ganti, latih, siap, mepet, main, remeh,...",5,positive
1,keren indonesia,"[keren, indonesia]",0,neutral
2,udah jadian aja deh jeje ma ririn 😭😭,"[udah, jadi, aja, deh, jeje, ma, ririn]",0,neutral
3,semangat timnas indonesia jangan pernah mereme...,"[semangat, timnas, indonesia, remeh, lawan, pe...",0,neutral
4,chemistry surya jegel emang the best👍,"[chemistry, surya, jegel, emang, the, best]",0,neutral
5,pucok,[pucok],0,neutral
6,selalu nunguin jegel nyanyi,"[nunguin, jegel, nyanyi]",1,positive
7,keren emang fiki spontanitasnya bikin vlognya ...,"[keren, emang, fiki, spontanitas, bikin, vlogn...",0,neutral
8,sumpah bangga bgt liat geek skrg😢😢,"[sumpah, bangga, bgt, liat, geek, skrg]",4,positive
9,is the definition of a combination of good loo...,"[is, the, definition, of, a, combination, of, ...",7,positive


**Because the previous labelling only examine score per word, we'll just label the data using library from huggingface**

In [20]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 124.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.7 MB/s eta 0:00:00


In [21]:
from transformers import pipeline
pretrained_name = "w11wo/indonesian-roberta-base-sentiment-classifier"

nlp = pipeline(
    "sentiment-analysis",
    model=pretrained_name,
    tokenizer=pretrained_name
)

comment_preprocessed = [str(i) for i in comments['Comment_Clean'].values]
comment_preprocessed = nlp(comment_preprocessed)

#comments['Sentiment'] = nlp(comments['Comment_Preprocessed'])

In [22]:
label_values = [d['label'] for d in comment_preprocessed]

comments['Sentiment2'] = label_values
comments

,Comment_Clean,Comment_Preprocessed,Polarity_Score,Sentiment,Sentiment2
0,efek ganti pelatih persiapan mepet dan juga pe...,"[efek, ganti, latih, siap, mepet, main, remeh,...",5,positive,negative
1,keren indonesia,"[keren, indonesia]",0,neutral,positive
2,udah jadian aja deh jeje ma ririn 😭😭,"[udah, jadi, aja, deh, jeje, ma, ririn]",0,neutral,positive
3,semangat timnas indonesia jangan pernah mereme...,"[semangat, timnas, indonesia, remeh, lawan, pe...",0,neutral,positive
4,chemistry surya jegel emang the best👍,"[chemistry, surya, jegel, emang, the, best]",0,neutral,positive
...,...,...,...,...,...
8350,buat pach sekarang emng memaksa xp gabakala...,"[pach, emng, paksa, xp, gabakalan, mvp, susah]",-7,negative,negative
8351,thank you rudy saya salut dengan kebanggaan ru...,"[thank, you, rudy, salut, bangga, rudy, presid...",21,positive,positive
8352,aransemen dc memang pancen wenakk gaaass mas d...,"[aransemen, dc, pancen, wenakk, gaaass, mas, den]",0,neutral,positive
8353,wonderful indonesia 🇮🇩❤️,"[wonderful, indonesia]",0,neutral,positive


In [23]:
comments.to_csv('comments_data_with_label.csv', index=False, header=True, index_label=True)

**Sentiment Analysis with Naive Bayes**

In [24]:
comments = pd.read_csv('comments_data_with_label.csv', lineterminator='\n')

In [28]:
comments

,Comment_Clean,Comment_Preprocessed,Polarity_Score,Sentiment,Sentiment2
0,efek ganti pelatih persiapan mepet dan juga pe...,"['efek', 'ganti', 'latih', 'siap', 'mepet', 'm...",5,positive,negative
1,keren indonesia,"['keren', 'indonesia']",0,neutral,positive
2,udah jadian aja deh jeje ma ririn 😭😭,"['udah', 'jadi', 'aja', 'deh', 'jeje', 'ma', '...",0,neutral,positive
3,semangat timnas indonesia jangan pernah mereme...,"['semangat', 'timnas', 'indonesia', 'remeh', '...",0,neutral,positive
4,chemistry surya jegel emang the best👍,"['chemistry', 'surya', 'jegel', 'emang', 'the'...",0,neutral,positive
...,...,...,...,...,...
8350,buat pach sekarang emng memaksa xp gabakala...,"['pach', 'emng', 'paksa', 'xp', 'gabakalan', '...",-7,negative,negative
8351,thank you rudy saya salut dengan kebanggaan ru...,"['thank', 'you', 'rudy', 'salut', 'bangga', 'r...",21,positive,positive
8352,aransemen dc memang pancen wenakk gaaass mas d...,"['aransemen', 'dc', 'pancen', 'wenakk', 'gaaas...",0,neutral,positive
8353,wonderful indonesia 🇮🇩❤️,"['wonderful', 'indonesia']",0,neutral,positive


In [26]:
# Map sentiment to numerical values

sentiment_encode = {'negative':0, 'neutral':1, 'positive':2}
y = comments['Sentiment2'].map(sentiment_encode).values

In [99]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words=None,ngram_range = (1,1),tokenizer = token.tokenize)
text_counts = cv.fit_transform(comments['Comment_Clean'].values.astype('U'))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [68]:
X=text_counts
y=comments['Sentiment2']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=30)

**Multinomial NB Modelling**

In [89]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import f1_score

MNB = MultinomialNB()
MNB.fit(X_train, y_train)

predicted = MNB.predict(X_test)
accuracy_score = metrics.accuracy_score(predicted, y_test)
f1_score = f1_score(predicted, y_test, average='weighted')
print(f1_score, accuracy_score)

0.7129507119583975 0.6935966487133453


In [105]:
import pickle
pickle.dump(cv, open("vector.pkl", "wb"))
pickle.dump(MNB, open('mnb_model.pkl', 'wb'))

**Sources:**
- https://www.kaggle.com/code/ankumagawa/sentimental-analysis-using-naive-bayes-classifier
- https://www.kaggle.com/code/tanmay111/youtube-comments-sentiment-analysis/notebook
- https://github.com/Kingki19/We-and-techa-Gemastik-Data-mining-2023
- https://www.kaggle.com/code/rifkyahmadsaputra/sentiment-analysis-online-lectures-in-indonesia
- https://www.kaggle.com/code/aderlaks/app-review-sentiment-analysis-indonesian-lexicon
- https://huggingface.co/w11wo/indonesian-roberta-base-sentiment-classifier